In [1]:
#Importing Necessary Libraries.

from PIL import Image
import numpy as np
import pandas as pd 
import os
import re
import cv2
from pathlib import Path

import matplotlib.pyplot as P
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4024)])
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
from scipy.stats import spearmanr as spr

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import glob2


tf.__version__

'2.2.0'

In [3]:
INPUT_TRUE = "/home/maia/Documents/Universidade/Universidade/UNIVERSIDADE/EXPERIMENTO/2EXPLAIN_NEW_VGG_CUSTOM/VGG/OCT/OUT/VGG-OCT-TRUE/*/*.png"
INPUT_FILES_TRUE = glob2.glob(INPUT_TRUE)

INPUT_RANDOM = "/home/maia/Documents/Universidade/Universidade/UNIVERSIDADE/EXPERIMENTO/2EXPLAIN_NEW_VGG_CUSTOM/VGG/OCT/OUT/VGG-OCT-RANDOM/*/*.png"
INPUT_FILES_RANDOM = glob2.glob(INPUT_RANDOM)


In [4]:
def image_generator(directory, res):
    test_datagen = ImageDataGenerator()
    image_generator = test_datagen.flow_from_directory(
            directory,
            target_size=(res, res),
            batch_size=1,
            color_mode = "grayscale",
            shuffle=False,
            class_mode='categorical')
    return image_generator

In [5]:
result_dict = {
    "puro_spearman":[],
    "pvalue" : [],
    "method" : []
}

result_dict = {
    "puro_spearman":[],
    "pvalue" : [],
    "method" : []
}


for im_TRUE, im_RANDOM in zip(INPUT_FILES_TRUE, INPUT_FILES_RANDOM):
    im_TRUELOAD = cv2.imread(im_TRUE)
    im_RANDOMLOAD = cv2.imread(im_RANDOM)
    
    puro_spearman = spr(im_TRUELOAD.flatten(), im_RANDOMLOAD.flatten())
    result_dict["method"].append(im_TRUE.split("/")[13])
    result_dict["puro_spearman"].append(puro_spearman.correlation)
    result_dict["pvalue"].append(puro_spearman.pvalue)

result_df = pd.DataFrame(result_dict)
result_df.to_csv("result.csv")